# Advanced Modeling and Simulation Course

### Instructor: Dr. M. Nedim ALPDEMİR

*This material is prepared as part of the EE592-Advanced Modeling And Simulation course given at Ankara Yıldırım Beyazıt University (AYBÜ).*

## Interactive Continuous Simulation Example with Visualization

###  Case Study : Mass Spring Damper Systems 

![image](mass-spr-damp.png)
<p><img src="//s0.wp.com/latex.php?latex=m+%5Cddot%7Bx%7D+%2B+c+%5Cdot%7Bx%7D+%2B+k+x+%3D+u&#038;bg=ffffff&#038;fg=000&#038;s=0" alt="m &#92;ddot{x} + c &#92;dot{x} + k x = u" title="m &#92;ddot{x} + c &#92;dot{x} + k x = u" class="latex" /></p>
<p>x : position of mass [m] at time t [s]<br />
m : mass [kg]<br />
c  : viscous damping coefficient [N s / m]<br />
k  : spring constant [N / m]<br />
u : force input [N]</p>
<p><span id="more-5"></span><br />

Depending on the values of m, c, and k, the system can be underdamped, overdamped or critically damped. For each case the behaviour of the system will be different.
In the case of the *Mass Spring Damper (MSD)*, we can see from the equation presented above, that the system is described by a 2nd order ODE. We would like to describe this system with two 1st order odes instead of one 2nd order ODE,
since having a set of first order ODEs allows us to arrange the equations into matrix form (state space) which is a convenient  representation that can be used for simulation using numerical methods!<br />

<h4>State Space Representation</h4>
<p>From the 2nd order ODE:</p>
$$m\ddot{x} + c\dot{x} + kx = u$$
<p>we can rename the dependent variable x and its first derivative as follows: </p>
$$x_1(t)=x(t)$$

$$x_2(t)=\dot{x}(t)=\dot{x_1}(t)$$

<p>x1 : position of the mass [m]<br />
x2 : derivative of x1 = velocity of the mass [m/s]<br />
    
Just from this change of variables we get the first 1st order ode.
    
$$\dot{x_1}(t)=x_2(t)$$
    
Substituting into the 2nd order ode and noting that:<br />
$$\dot{x_2}(t)=\ddot{x}(t)$$
    
we get:<br />

$$\dot{x_2} + c {x_2} + k x_1 = u$$

which leads to our second 1st order ode:<br />

$$\dot{x_2} = - (c/m) {x_2} - (k/m) x_1 + (1/m) u$$

<p>Now, we can put the set of equations into matrix form as:</p>
<div lang="latex">
<img src="//s0.wp.com/latex.php?latex=%5Cbegin%7Bbmatrix%7D+%5Cdot%7Bx_1%7D%5C%5C+%5Cdot%7Bx_2%7D+%5Cend%7Bbmatrix%7D+%3D+%5Cbegin%7Bbmatrix%7D+0+%26+1%5C%5C+-+k%2Fm+%26-c%2Fm+%5Cend%7Bbmatrix%7D+%5Cbegin%7Bbmatrix%7D+x_1%5C%5Cx_2+%5Cend%7Bbmatrix%7D+%2B+%5Cbegin%7Bbmatrix%7D+0%5C%5C1%2Fm+%5Cend%7Bbmatrix%7D+u&#038;bg=ffffff&#038;fg=000&#038;s=0" alt="&#92;begin{bmatrix} &#92;dot{x_1}&#92;&#92; &#92;dot{x_2} &#92;end{bmatrix} = &#92;begin{bmatrix} 0 &amp; 1&#92;&#92; - k/m &amp;-c/m &#92;end{bmatrix} &#92;begin{bmatrix} x_1&#92;&#92;x_2 &#92;end{bmatrix} + &#92;begin{bmatrix} 0&#92;&#92;1/m &#92;end{bmatrix} u" title="&#92;begin{bmatrix} &#92;dot{x_1}&#92;&#92; &#92;dot{x_2} &#92;end{bmatrix} = &#92;begin{bmatrix} 0 &amp; 1&#92;&#92; - k/m &amp;-c/m &#92;end{bmatrix} &#92;begin{bmatrix} x_1&#92;&#92;x_2 &#92;end{bmatrix} + &#92;begin{bmatrix} 0&#92;&#92;1/m &#92;end{bmatrix} u" class="latex" />
</div>
<p>with the output equation (assuming we are interested in the postion of the mass):</p>
<div lang="latex">
<img src="//s0.wp.com/latex.php?latex=y+%3D+%5Cbegin%7Bbmatrix%7D+%7B1%7D%26%7B0%7D+%5Cend%7Bbmatrix%7D+%5Cbegin%7Bbmatrix%7D+%7Bx_1%7D%5C%5C+%7Bx_2%7D+%5Cend%7Bbmatrix%7D&#038;bg=ffffff&#038;fg=000&#038;s=0" alt="y = &#92;begin{bmatrix} {1}&amp;{0} &#92;end{bmatrix} &#92;begin{bmatrix} {x_1}&#92;&#92; {x_2} &#92;end{bmatrix}" title="y = &#92;begin{bmatrix} {1}&amp;{0} &#92;end{bmatrix} &#92;begin{bmatrix} {x_1}&#92;&#92; {x_2} &#92;end{bmatrix}" class="latex" />
</div>

## Simulation Experiment

* implement a one step solver for the above differential equations
* use Vpython to generate geometric objects and a virtual scene to represent the MSD systems defined above  
* map the solution for the displacement of the mass (i.e. variable x) to the motion of the geometric object representing the mass, connected to the end of a damped spring
* implement necessary GUI components to allow the user to interact wth the system by changing:
    - the external force,
    - the damping coefficient,
    - the spring constant, and
    - the mass

In [7]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import ode
from vpython import *
import ipywidgets as wd
 
class MSDSystem(): 

    def __init__(self):

        self.h0 = [0,0]
        self.t0 = 0.0
        self.k = 2.0    # spring constant
        self.mass = 1.0   # mass 
        self.damp = 0.25 # damping factor
        self.du = 2.0    # change in u (external force)
        self.wigt = 0.0
        self.l = 6
        self.wh = 2
        def intFunc(t, x, du):
            y = x[0]
            dydt = x[1]
            dy2dt2 = (- self.damp/self.mass*dydt - (self.k/self.mass) * y + (1/self.mass)*du)
            return [dydt,dy2dt2]
        self.integrator = ode(intFunc)
        self.integrator.set_initial_value(self.h0, self.t0).set_f_params(self.du)
        self.integrator.set_integrator("dopri5")
        dt = 0.05
        self.wall = box(pos=vector(0,self.wh/2,-1), size=vector(0.1,self.wh,2), color=color.red)
        self.base = box(pos=vector(self.l/2,0,-1), size=vector(self.l,0.1,2), color=color.red) 
        self.box = box(pos=vector(1.8,0.4,-1), size=vector(0.8,0.8,0.8), color=color.blue) 
        self.spring = helix(pos=vector(0.0,0.5,-1), radius=0.2, axis=vector(1.8,0,0), color=color.orange, thickness=.05,coils=14)
        self.box_po_x = self.box.pos.x
        self.xi_1 = 0
    
    def reset(self):
        self.integrator.set_initial_value(self.h0, self.t0).set_f_params(self.du)
        
    def wiggle(self, dx):
        #self.wigt += dt
        #self.box.pos.x = self.box_po_x + 0.5*sin(-4*self.wigt)
        self.box.pos.x += dx
        self.spring.length = self.box.pos.x-self.spring.pos.x  - 0.4
        
    def step(self, t):    
        self.integrator.integrate(t)
        x = self.integrator.y[0] 
        return x
   
scene = canvas() # This is needed in Jupyter notebook and lab to make programs easily rerunnable
scene.title = "A 3D Display of Mass Spring Damper System in action"
scene.width = 740
scene.height = 400
#scene.range = 5
scene.background = color.gray(0.9)
scene.center = vector(3,0,0)
scene.forward = vector(0.5,-0.5,-1)
scene.autoscale = False
solver = MSDSystem()
running = True
t = 0.0
xi_1 = 0.0
dt = 0.01

m_sl = wd.FloatSlider(description='Mass (m):', min=0.2, max=4, step=0.2, value=2)
d_sl = wd.FloatSlider(description='Damping Coefficient (c):', min=0.0, max=1.9, step=0.1, value=0.25)
s_sl = wd.FloatSlider(description='Spring Constant (k):', min=0.5, max=5, step=0.5, value=1)
u_sl = wd.FloatSlider(description='External Force (du) :', min=1, max=10, step=0.5, value=2)
sl_container = wd.VBox(children=[m_sl, d_sl, s_sl, u_sl])
display(sl_container)

ps_btn = wd.Button(description = 'Pause')
def ps_btn_handler(s):
    global running
    running = not running
    if s.description == 'Run': 
        s.description = 'Pause'
        print("now running ...")
    else: 
        s.description = 'Run'
        print("now paused ...")
ps_btn.on_click(ps_btn_handler)

res_btn = wd.Button(description='Reset')
def res_btn_handler(s):  
    global t
    global xi_1
    global solver
    print("resetting..")
    t = 0.0
    solver.reset()
res_btn.on_click(res_btn_handler)

btn_container = wd.HBox(children=[res_btn,ps_btn])
display(btn_container)

import asyncio
async def periodic():
    global solver
    global t
    global dt
    global xi_1
    while True:
        await asyncio.sleep(0)
        rate(1/dt)
        if running:
            t += dt 
            solver.k = s_sl.value
            solver.mass = m_sl.value
            solver.damp = d_sl.value
            solver.du = u_sl.value
            x = solver.step(t)
            dx = x - xi_1
            solver.wiggle(dx)
            xi_1 = x

loop = asyncio.get_event_loop()
task = loop.create_task(periodic())


<IPython.core.display.Javascript object>

resetting..
resetting..
